In [9]:
from qdrant_client import QdrantClient, models

In [10]:
client = QdrantClient(url="http://localhost:6333")

In [11]:
if client.collection_exists(collection_name="collection0"):
    client.delete_collection(collection_name="collection0")

client.create_collection(
    collection_name="collection0",
    vectors_config=models.VectorParams(size=4, distance=models.Distance.DOT),
)

client.upsert(
    collection_name="collection0",
    points=models.Batch(
        ids=[1, 2, 3, 4, 5, 6],
        payloads=[
            {"color": "red", "city": "London", "age": 24},
            {"color": "green", "city": "Moscow", "age": 34},
            {"color": "blue", "city": "Moscow", "age": 13},
            {"color": "green", "city": "London", "age": 56},
            {"color": "red", "city": "Berlin", "age": 88},
            {"color": "blue", "city": "London", "age": 8},
        ],
        vectors=[
            [0.1, 0.1, 0.1, 0.1],
            [0.2, 0.2, 0.2, 0.2],
            [0.3, 0.3, 0.3, 0.3],
            [0.4, 0.4, 0.4, 0.4],
            [0.5, 0.5, 0.5, 0.5],
            [0.6, 0.6, 0.6, 0.6],
        ],
    ),
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [12]:
filter_ = models.Filter(
    must=[
        models.FieldCondition(
            key="city",
            match=models.MatchValue(
                value="London",
            ),
        )
    ]
)

search_queries = [
    models.SearchRequest(vector=[0.2, 0.1, 0.9, 0.7], filter=filter_, limit=3, with_payload=True),
    models.SearchRequest(vector=[0.5, 0.3, 0.2, 0.3], filter=filter_, limit=1, with_vector=True),
]

# Perform multiple searches in a collection mitigating network overhead
client.search_batch(collection_name="collection0", requests=search_queries)

[[ScoredPoint(id=6, version=0, score=1.1400001, payload={'age': 8, 'city': 'London', 'color': 'blue'}, vector=None, shard_key=None, order_value=None),
  ScoredPoint(id=4, version=0, score=0.76, payload={'age': 56, 'city': 'London', 'color': 'green'}, vector=None, shard_key=None, order_value=None),
  ScoredPoint(id=1, version=0, score=0.19, payload={'age': 24, 'city': 'London', 'color': 'red'}, vector=None, shard_key=None, order_value=None)],
 [ScoredPoint(id=6, version=0, score=0.78000003, payload=None, vector=[0.6, 0.6, 0.6, 0.6], shard_key=None, order_value=None)]]